
              # # auto encoders
                                                        

#####  -  downloading the data  through the link below  
https://drive.google.com/drive/folders/1eB1OUAKrl4KXvgrNbmzRqywZUCqBuga6?usp=drive_link
##### -  motivation to use npy format to store your data
https://towardsdatascience.com/what-is-npy-files-and-why-you-should-use-them-603373c78883

The folder contains 2 subfolders, 'trainingset' and 'testingset'
```
trainingset/
├── train.npy

testingset/
├── test.npy
...
```

### import packages 

In [4]:
import numpy as np
import random
import torch

from torch.utils.data import DataLoader
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import torchvision.transforms as transforms

from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models

from torch.optim import Adam, AdamW

from sklearn.cluster import MiniBatchKMeans
from scipy.cluster.vq import vq, kmeans

from tqdm import tqdm
import pandas as pd

import pdb  # use pdb.set_trace() to set breakpoints for debugging

### loading data 

In [5]:
train_all = np.load('..\\train.npy')
test_all = np.load('..\\test.npy')

train = train_all[0:1000, :, :, :] ## using subdataset for testing codes 
test = test_all[0:500, :, :, :] ## using subdataset for testing codes
print (train_all.shape)
print(test_all.shape)
print (train.shape)
print(test.shape)

(140001, 64, 64, 3)
(19999, 64, 64, 3)
(1000, 64, 64, 3)
(500, 64, 64, 3)


## Model and loss
refer to 
[1] https://github.com/L1aoXingyu/pytorch-beginner
[2] https://github.com/jellycsc/PyTorch-CIFAR-10-autoencoder/

In [6]:
class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(48, 96, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  
            nn.ReLU(),
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
        
    ########################################################################################
    
    ###### write the network forward function here and return the output "x" ######
    def forward(self, x):

        
        return (x)
            
    ########################################################################################    

### Dataset module

The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].

In [7]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """

    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)

        self.transform = transforms.Compose([
            transforms.Lambda(lambda x: x.to(torch.float32)),
            transforms.Lambda(lambda x: 2. * x / 255. - 1.),
            # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ])

    def __getitem__(self, index):
        x = self.tensors[index]

        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

## Training 
##### Initialize
- hyperparameters
- dataloader
- model
- optimizer & loss

In [8]:
num_epochs = 15 # 50
batch_size = 1000 # medium: smaller batchsize 10000 by default
learning_rate = 1e-3
model_type = 'cnn'
# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model

model = conv_autoencoder().cpu()
## model = conv_autoencoder().cuda() ## only if you have a GPU


#######  Loss and optimizer ####### 

## TODO: define mean square error loss function here from the torch libary

criterion =    

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

############################################################################################## 

## Training loop

In [9]:
best_loss = np.inf
model.train()
tqdm_train = tqdm(range(num_epochs))

for epoch in tqdm_train:
    tot_loss = list()
    for data in train_dataloader:

        # ===================loading=====================

        img = data.float().cpu()
        #img = data.float().cuda() # for GPU case

        # ===================forward=====================
        output = model(img)
        
        ####################################################
        # TODO: calculate the loss value between img and its output based on the defined "criterion" above

        loss = 

        ###################################################
        
        
        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    # ===================log========================
    
    print('Epoch: ', epoch, 'loss: ', mean_loss)
    

    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                                                                       
 0/15           -             -                                                                                         
Description   0.0% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 1/15   00:00:02<00:00:30  0.46it/s  1/15   0.3856                                                                      
Description   6.7% |██████                                                                                             | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 2/15   00:00:04<00:00:26  0.49it/s  2/15   0.3844                                                                      
Description  13.3% |█████████████ 

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [10]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=0) # num_workers = 1 if you have GPUs
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = 'last_model_cnn.pt'
model = torch.load(checkpoint_path)
model.eval()

# prediction file
out_file = 'PREDICTION_FILE.csv'

In [11]:
anomality = list()
with torch.no_grad():
  for i, data in enumerate(test_dataloader):

        img = data.float().cpu()
        output = model(img)
        loss = eval_loss(output, img).sum([1, 2, 3])
        anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['Predicted'])
# saving the results in the csv file
df.to_csv(out_file, index_label = 'Id') 
## the results are reconstruction errors referred as anomaly scores

# Extra work:

This assigment is just to show the pipeline for anomaly  detection with autoencoder using a very simple encoder-decoder architecture.  it would be better to use the other classical networks for feature extraction and construction.

TODO: change the encoder and decoder in the class of conv_autoencoder